In [ ]:
import pandas as pd

df = pd.read_csv("mrbilit_dataset.csv")
display(df.head())

,Created,CancelTime,DepartureTime,BillID,TicketID,ReserveStatus,UserID,Male,Price,CouponDiscount,...,Domestic,VehicleType,VehicleClass,TripReason,Vehicle,Cancel,HashPassportNumber_p,HashEmail,BuyerMobile,NationalCode
0,2022-07-26 13:33:20.457,NaN,2022-07-26 16:30:00,38428546,7445571.0,3,NaN,True,1180000.0,0.0,...,1,VIPمانیتوردار-شارژراختصاصی تخت شو مارال (جدید)...,True,Work,Bus,0,NaN,NaN,302222356019,330024570
1,2022-10-27 23:07:01.837,2022-10-27 23:26:39.070,2022-10-29 09:45:00,39768762,7762719.0,5,NaN,False,1050000.0,0.0,...,1,classicus 2+2,True,Int,Bus,1,NaN,NaN,900764168521,995520696
2,2022-09-12 11:01:13.607,NaN,2022-10-03 18:35:00,39128001,2327596.0,5,800398.0,False,4674000.0,0.0,...,1,فوکر 100,False,Int,Plane,0,NaN,1c44d7a76b52341fa12dcfa993138576befcc9ebf01d14...,749804783291,979382950
3,2022-08-08 17:43:35.840,NaN,2022-08-08 22:30:00,38606546,7495440.0,3,NaN,True,1200000.0,0.0,...,1,VIPدرسا+مانیتوردار+شارژراختصاصی+پذیرایی,True,Work,Bus,0,NaN,NaN,781396205677,911237229
4,2022-11-01 15:12:56.823,NaN,2022-11-03 11:30:00,39822185,2356902.0,5,NaN,True,6222000.0,0.0,...,1,NaN,False,Work,Plane,0,NaN,bb38b345aec02255e31d178492907175c5984f2a1f5b59...,524576220177,727496008


In [ ]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101017 entries, 0 to 101016
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Created               101017 non-null  object 
 1   CancelTime            15312 non-null   object 
 2   DepartureTime         101017 non-null  object 
 3   BillID                101017 non-null  int64  
 4   TicketID              101017 non-null  float64
 5   ReserveStatus         101017 non-null  int64  
 6   UserID                42502 non-null   float64
 7   Male                  101017 non-null  bool   
 8   Price                 101017 non-null  float64
 9   CouponDiscount        101017 non-null  float64
 10  From                  101017 non-null  object 
 11  To                    101017 non-null  object 
 12  Domestic              101017 non-null  int64  
 13  VehicleType           93339 non-null   object 
 14  VehicleClass          62666 non-null   object 
 15  

None

Based on the initial exploration of the dataset, some columns appear less useful due to a high number of missing values:

*   `CancelTime`
*   `UserID`
*   `HashPassportNumber_p`
*   `HashEmail`

Additionally, columns like `BuyerMobile`, `NationalCode`, `BillID`, and `TicketID` are identifiers. While they are complete (no missing values), their usefulness depends on the specific analysis goals. They are essential for analyzing individual transactions or buyer behavior but may not be directly needed for aggregated analysis.

In [ ]:
vehicle_counts = df['Vehicle'].value_counts()
import plotly.express as px

fig = px.bar(vehicle_counts,
             x=vehicle_counts.index,
             y=vehicle_counts.values,
             labels={'x': 'Vehicle Type', 'y': 'Count'},
             title='Count of Each Vehicle Type')
fig.show()

In [ ]:
import plotly.express as px

df_clipped = df.copy()
df_clipped['Price'] = df_clipped['Price'].clip(upper=20000000)

fig = px.histogram(df_clipped,
                   x='Price',
                   color='Vehicle',
                   title='Price Distribution by Vehicle Type (Clipped at 20M)')
fig.show()

In [ ]:
cancellation_rate = df['Cancel'].mean() * 100
print(f"Overall Cancellation Rate: {cancellation_rate:.2f}%")

Overall Cancellation Rate: 15.15%


In [ ]:
price_and_loss_per_vehicle = df.groupby('Vehicle').agg(
    total_price=('Price', 'sum'),
    total_cancelled_price=('Price', lambda x: x[df.loc[x.index, 'Cancel'] == 1].sum()),
    total_trips=('BillID', 'count'),
    cancellation_rate=('Cancel', 'mean')
).reset_index()

price_and_loss_per_vehicle.columns = ['Vehicle', 'Total Price (Millions)', 'Loss from Cancel (Millions)', 'Total Trips', 'Cancellation Rate']

# Convert price and loss to millions
price_and_loss_per_vehicle['Total Price (Millions)'] = price_and_loss_per_vehicle['Total Price (Millions)'] / 1000000
price_and_loss_per_vehicle['Loss from Cancel (Millions)'] = price_and_loss_per_vehicle['Loss from Cancel (Millions)'] / 1000000

print("Summary Statistics per Vehicle Type:")
display(price_and_loss_per_vehicle)

Summary Statistics per Vehicle Type:


,Vehicle,Total Price (Millions),Loss from Cancel (Millions),Total Trips,Cancellation Rate
0,Bus,73383.623363,10287.558817,48833,0.134213
1,InternationalPlane,40317.027140,3344.233590,798,0.082707
2,Plane,127879.306551,16213.678363,13035,0.127273
3,Train,82653.359766,16327.950592,38351,0.183098


In [ ]:
# Filter the DataFrame for train trips
train_df = df[df['Vehicle'] == 'Train']

# Analyze 'To' cities
to_city_counts = train_df['To'].value_counts()
print("Top 5 'To' Cities for Train Trips:")
display(to_city_counts.head())
print(f"\nNumber of unique 'To' cities for Train Trips: {to_city_counts.shape[0]}")

# Analyze 'From' cities
from_city_counts = train_df['From'].value_counts()
print("\nTop 5 'From' Cities for Train Trips:")
display(from_city_counts.head())
print(f"\nNumber of unique 'From' cities for Train Trips: {from_city_counts.shape[0]}")

Top 5 'To' Cities for Train Trips:


,count
To,
تهران,13335
مشهد,10372
قم,2569
یزد,1130
کرمان,944



Number of unique 'To' cities for Train Trips: 74

Top 5 'From' Cities for Train Trips:


,count
From,
تهران,14272
مشهد,8886
قم,2406
یزد,1329
کرمان,1004



Number of unique 'From' cities for Train Trips: 73


# Task
Analyze the "To" and "From" columns in the DataFrame where 'vehicle' is 'train'. Identify the top 19 cities based on the combined counts from both columns. Create new columns 'From_Category' and 'To_Category' where each city is labeled as one of the top 19 or 'Other'. Display the value counts for the new categorical columns.

## Identify top cities

### Subtask:
Determine the top 19 cities based on the combined counts from 'From' and 'To' columns.


**Reasoning**:
Combine the 'From' and 'To' columns, calculate value counts, and select the top 19 cities.



In [ ]:
combined_cities = pd.concat([train_df['From'], train_df['To']])
city_counts = combined_cities.value_counts()
top_cities = city_counts.head(19).index.tolist()
print("Top 19 Cities (Combined 'From' and 'To'):")
print(top_cities)

Top 19 Cities (Combined 'From' and 'To'):
['تهران', 'مشهد', 'قم', 'یزد', 'کرمان', 'اهواز', 'زنجان', 'بندرعباس', 'اصفهان', 'تبریز', 'شیراز', 'کرج', 'سمنان', 'رشت', 'شاهرود', 'قزوین', 'ساری', 'زاهدان', 'میانه']


## Categorize cities

### Subtask:
Create a function or mapping to categorize each city as one of the top 19 or 'Other'.


**Reasoning**:
Define a function to categorize cities based on whether they are in the top 19 cities list.



In [ ]:
def categorize_city(city):
    """Categorizes a city as one of the top 19 or 'Other'."""
    if city in top_cities:
        return city
    else:
        return 'Other'

## Create new columns

### Subtask:
Apply the categorization to create new 'From_Category' and 'To_Category' columns in the DataFrame.


**Reasoning**:
Apply the categorization function to the 'From' and 'To' columns of the train_df DataFrame to create the new categorical columns as required by the subtask.



In [ ]:
train_df['From_Category'] = train_df['From'].apply(categorize_city)
train_df['To_Category'] = train_df['To'].apply(categorize_city)
display(train_df[['From', 'From_Category', 'To', 'To_Category']].head())

/tmp/ipython-input-3014501249.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-3014501249.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,From,From_Category,To,To_Category
6,مشهد,مشهد,تهران,تهران
7,تهران,تهران,قم,قم
9,سرخس,Other,مشهد,مشهد
11,قم,قم,تهران,تهران
12,بجستان,Other,تهران,تهران


**Reasoning**:
Display the value counts for the newly created categorical columns to verify the categorization.



In [ ]:
print("\nValue Counts for 'From_Category':")
display(train_df['From_Category'].value_counts())
print("\nValue Counts for 'To_Category':")
display(train_df['To_Category'].value_counts())


Value Counts for 'From_Category':


,count
From_Category,
تهران,14272
مشهد,8886
Other,2745
قم,2406
یزد,1329
کرمان,1004
اهواز,900
زنجان,860
اصفهان,860



Value Counts for 'To_Category':


,count
To_Category,
تهران,13335
مشهد,10372
قم,2569
Other,2479
یزد,1130
کرمان,944
اهواز,902
زنجان,878
تبریز,837


## Verify categorization

### Subtask:
Display the value counts for the new categorical columns to ensure the categorization was applied correctly.


**Reasoning**:
Display the value counts for the new categorical columns to ensure the categorization was applied correctly.



In [ ]:
print("\nValue Counts for 'From_Category':")
display(train_df['From_Category'].value_counts())
print("\nValue Counts for 'To_Category':")
display(train_df['To_Category'].value_counts())


Value Counts for 'From_Category':


,count
From_Category,
تهران,14272
مشهد,8886
Other,2745
قم,2406
یزد,1329
کرمان,1004
اهواز,900
زنجان,860
اصفهان,860



Value Counts for 'To_Category':


,count
To_Category,
تهران,13335
مشهد,10372
قم,2569
Other,2479
یزد,1130
کرمان,944
اهواز,902
زنجان,878
تبریز,837


## Summary:

### Data Analysis Key Findings

*   The top 19 cities based on the combined counts from the 'From' and 'To' columns in the train data were identified.
*   New columns, 'From\_Category' and 'To\_Category', were created in the DataFrame where each city is categorized as one of the top 19 cities or 'Other'.
*   The value counts for 'From\_Category' and 'To\_Category' were successfully displayed, showing the distribution of cities within the defined categories.

### Insights or Next Steps

*   Further analysis could explore the travel patterns between the top 19 cities and how they compare to travel involving 'Other' locations.
*   This categorization can be useful for simplifying analysis and visualization of travel data by focusing on the most frequent locations.


In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Calculate the number of trips between each pair of 'From_Category' and 'To_Category'
trip_counts = train_df.groupby(['From_Category', 'To_Category']).size().reset_index(name='Number of Trips')

# Add a small value before taking the log to handle zero counts
trip_counts['Log_Number_of_Trips'] = np.log1p(trip_counts['Number of Trips'])

# The trip_counts DataFrame is already calculated and contains 'From_Category', 'To_Category', and 'Number of Trips'

# Create a list of the unique 'From_Category' and 'To_Category' for the axes
y_categories = trip_counts['From_Category'].unique()
x_categories = trip_counts['To_Category'].unique()

# Create a pivot table to get the data in the correct format for go.Heatmap
heatmap_data = trip_counts.pivot(index='From_Category', columns='To_Category', values='Log_Number_of_Trips').fillna(0)
hover_text_data = trip_counts.pivot(index='From_Category', columns='To_Category', values='Number of Trips').fillna(0)

# Reindex the heatmap data and hover text data to ensure consistent order of categories
heatmap_data = heatmap_data.reindex(index=y_categories, columns=x_categories)
hover_text_data = hover_text_data.reindex(index=y_categories, columns=x_categories)


# Create the heatmap using go.Heatmap
fig = go.Figure(data=go.Heatmap(
                   z=heatmap_data.values,
                   x=heatmap_data.columns,
                   y=heatmap_data.index,
                   text=hover_text_data.values, # Use 'text' for go.Heatmap hover
                   hoverinfo='text', # Set hoverinfo to 'text' to display the custom text
                   colorscale='Viridis')) # You can change the colorscale here

fig.update_layout(
    title='Number of Train Trips Between Categorized Cities (Log Scale)',
    xaxis_title='To_Category',
    yaxis_title='From_Category'
)

fig.show()

In [ ]:
cancellation_rate_by_from = train_df.groupby('From_Category').agg(
    total_trips=('BillID', 'count'),
    cancellation_rate=('Cancel', 'mean')
).reset_index()

cancellation_rate_by_from.columns = ['From_Category', 'Total Trips', 'Cancellation Rate']

print("Cancellation Rate and Total Trips by From_Category:")
display(cancellation_rate_by_from.sort_values(by='Cancellation Rate', ascending=False))


Cancellation Rate and Total Trips by From_Category:


,From_Category,Total Trips,Cancellation Rate
2,اهواز,900,0.352222
1,اصفهان,860,0.305814
11,شاهرود,343,0.282799
9,ساری,247,0.230769
10,سمنان,416,0.228365
15,مشهد,8886,0.214832
19,یزد,1329,0.211437
7,زاهدان,279,0.186380
8,زنجان,860,0.181395
0,Other,2745,0.180328


In [ ]:
import plotly.express as px

fig = px.bar(cancellation_rate_by_from.sort_values(by='Cancellation Rate', ascending=False),
             x='From_Category',
             y='Cancellation Rate',
             title='Cancellation Rate by Origin City Category (Train Trips)',
             labels={'From_Category': 'Origin City Category', 'Cancellation Rate': 'Cancellation Rate'})

fig.update_layout(yaxis_tickformat=".1%") # Format y-axis as percentage

fig.show()